[Chain How to Chaining](https://python.langchain.com/docs/modules/chains/)

# 什么chains

组件可能包括但不限于: **prompt**, **memory**, **retrieval**, **model**，甚至可以是chain本身.

# Build-In chains example: LLMChain

> 接收prompt-template, llm, output_parser

> The legacy approach is to use the `Chain` interface. The updated approach is to use the [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/expression_language/). <br>
> 传统方式是`Chain`接口, 新的方式是使用**LCEL**

> 构建新的链组合,我们建议使用**LCEL**

> Chain本身也能在LCEL中使用, 二者并不冲突

## LCEL approach

> As a simple and common example, we can see what it's like to combine a prompt, model and output parser:<br>
> 在简单的案例中,我们组合prompt, model, he output parser

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

In [3]:
model = ChatOpenAI()

In [4]:
prompt = ChatPromptTemplate.from_messages([
    ('system', "You're a very knowledgeable historian who provides accurate and eloquent answers to historical questions."),
    ("human", "{question}"),
     ])

In [5]:
runnable = prompt | model | StrOutputParser()

In [6]:
for chunk in runnable.stream({"question": "How did Mansa Musa accumulate his wealth?"}):
    print(chunk, end="", flush=True)

Mansa Musa, the renowned ruler of the Mali Empire in the 14th century, amassed his immense wealth through a combination of factors. The primary source of his riches was the vast gold reserves within his empire. Mali, located in West Africa, was a region abundant in gold mines, and Mansa Musa had control over these lucrative resources.

Under his rule, Mali became a significant producer and exporter of gold, which was in high demand across the trans-Saharan trade routes. Mansa Musa effectively monopolized the gold trade, ensuring that the majority of the precious metal flowed through his empire. This allowed him to exert control over its value and regulate its distribution, further augmenting his wealth.

Furthermore, Mansa Musa expanded Mali's influence and control over other key trade routes, such as salt and copper. The empire's strategic location enabled it to profit from the trade between North Africa, the Mediterranean, and sub-Saharan Africa. By taxing and regulating these trade 

## [Legacy] Chain interface

Chain的接口很简单

```
class Chain(BaseModel, ABC):
    """Base interface that all chains should implement."""

    memory: BaseMemory
    callbacks: Callbacks

    def __call__(
        self,
        inputs: Any,
        return_only_outputs: bool = False,
        callbacks: Callbacks = None,
    ) -> Dict[str, Any]:
        ...
```

> We can recreate the LCEL runnable we made above using the built-in LLMChain:<br>
> 我们使用LLMChain重构上述的LCEL

In [7]:
from langchain.chains import LLMChain

In [8]:
chain = LLMChain(llm=model, prompt=prompt, output_parser=StrOutputParser())

In [10]:
chain.run(question="一般的积累财富的方法有哪些")

'积累财富的方法有很多种，以下是一些常见的方法：\n\n1. 节约和理性消费：通过控制开支和避免不必要的消费，将可用资金存入储蓄账户或投资。\n\n2. 定期储蓄：将一部分收入定期存入银行或其他金融机构的储蓄账户，以积累利息和逐渐增加财富。\n\n3. 投资：通过购买股票、债券、房地产或其他资产来增加财富。投资的选择应该根据个人风险承受能力和财务目标来确定。\n\n4. 创业：创业可以为个人带来巨大的财富增长机会，但也伴随着风险。成功的创业者需要有创新精神、市场洞察力和管理能力。\n\n5. 教育和技能培训：通过提高自己的技能和知识水平，增加在职场中的竞争力，从而获得更高的薪资和职业机会。\n\n6. 多样化收入来源：通过拥有多个收入来源，如兼职工作、投资收益或出租房产，可以减少财务风险，并增加财富积累的机会。\n\n7. 遗产规划：制定财务计划，包括编制遗嘱、购买保险和合理规划财产分配，以确保个人财富能够在世代传承。\n\n请注意，每个人的财务目标和情况都不同，因此选择适合自己的方法是非常重要的。此外，财富积累需要时间和耐心，没有一种快速致富的捷径。'

## 注意事项

<font color=blue>只有在chain的output_keys只有一个元素时, 才可以使用run方法.</font> <br>
<font color=blue>对于只有一个输入变量的情况下, 才直接输入字符串, 否则输入dict类型</font> <br>

# 源码分析

![UML](./imgs/UML.png)

## 一: Chain

### 属性

Chain类的几个关键属性:
1. memory: **调试及传参用**.可选的`BaseManager`对象,默认为None. Memory类在每个链条开始和结束时被调用. **开始时Memory加载变量**并**在链中传递它们**, 最后,它**保存返回任何返回的变量**<br>
2. callbacks: 回调管理器列表, 默认None, 在chain的生命周期中,会始终调用**回调管理器**<br>
3. callback_manager: 已弃用, 请使用`callbacks`<br>
4. verbose: 默认值`langchain.__init__.verbose=False`<br>
5. ~~tags:~~ <br>
6. ~~metadata:~~ <br>


### 方法

1. stateful : Chain + Memory , 意味着Chain可以在多次运行之间记住并持久化其数据<br>
2. Observable:  将Callbacks传递给Chain, 可以执行额外的功能. 例如component调用的主流程之外进行日志记录. <font color=blue>这让你更好地监控和管理链条的运行过程</font><br>
3. Composable: 自由的设计和构建复杂的流程

#### **主要方法**

1. `__call__`: 输入字典, 输出字典<br>
2. `run`: 这是一个便捷方法, <font color=blue>输入可以是字符串或者字典</font>, 但`output_keys`只有一个, 且返回值是`str`

#### **抽象方法**

1. 输入中预期的键
```
@property
@abstractmethod
def input_keys(self) -> List[str]:
    """Input keys this chain expects."""
```

2. 输出中预期的键
```
@property
@abstractmethod
def output_keys(self) -> List[str]:
    """Output keys this chain expects."""
```

3. `_call`: 此方法<font color=blue>此方法执行链</font>, 私有方法, 不面向用户. 只在`Chain.call`内部调用.<br>
  `_acall`, 上述方法的异步

4. `_chain_type(self) -> str`: 返回chain的类型名称

#### **非抽象方法**

```python
def _validate_inputs(self, inputs: Dict[str, Any]) -> None:
       """Check that all inputs are present."""
       missing_keys = set(self.input_keys).difference(inputs)
       if missing_keys:
           raise ValueError(f"Missing some input keys: {missing_keys}")

def _validate_outputs(self, outputs: Dict[str, Any]) -> None:
    missing_keys = set(self.output_keys).difference(outputs)
    if missing_keys:
        raise ValueError(f"Missing some output keys: {missing_keys}")
```

<font color=blue>检查传入的 inputs 字典中的所有键是否都存在于 self.input_keys 中。如果有缺失的键，就抛出一个 ValueError 异常，并提示缺少哪些键</font><br>
<font color=blue>检查传入的 outputs 字典中的所有键是否都存在于 self.output_keys 中。如果有缺失的键，也会抛出一个 ValueError 异常，并提示缺少哪些键</font>

`prep_inputs` 和 `prep_outputs`: 
这两个方法<font color=blue>负责验证和准备链的输入和输出</font>包括: **从内存中添加输入、保存运行信息到内存**

1. 二者调用了`_validate_outputs`和`_validate_inputs`<br>
2. 公共方法, 负责验证, 准备Chain类的输入, 并根据需要从类内部的**运行时存储Memory**获取附加输入.<br>
3. 返回字典类型:**包含原始输入以及来自内存Memory的附加输入**

```python
>> 责验证和准备 Chain 类的输入
def prep_inputs(self, inputs: Union[Dict[str, Any], Any]) -> Dict[str, str]:
    """Validate and prep inputs."""
    >> 如果`inputs`不是字典类型, 该方法会假定Chain只接受一个输入参数.
    if not isinstance(inputs, dict):
        _input_keys = set(self.input_keys)
        >> 在此情况下,会检查是否有来自内存Memeory的输入, 有则更新`_input_keys`, 
        if self.memory is not None:
            # If there are multiple input keys, but some get set by memory so that
            # only one is not set, we can still figure out which key it is.
            _input_keys = _input_keys.difference(self.memory.memory_variables)
        >> 然后,如果`_input_keys`的长度大于1, 则抛出ValueError异常. 因为已经默认输入参数为1个了
        if len(_input_keys) != 1:
            raise ValueError(
                f"A single string input was passed in, but this chain expects "
                f"multiple inputs ({_input_keys}). When a chain expects "
                f"multiple inputs, please call it by passing in a dictionary, "
                "eg `chain({'foo': 1, 'bar': 2})`"
            )
        >> 接下来,将唯一的输入数据转化为**字典类型的结构**
        inputs = {list(_input_keys)[0]: inputs}
    
    if self.memory is not None:
        >> 如果Chain类具有内存功能, 那么它会从内存中加载输入数据并添加到输入字典中, 
        >> 以确保所有必要的输入都已准备就绪
        external_context = self.memory.load_memory_variables(inputs)
        inputs = dict(inputs, **external_context)
    >> 然后，调用 _validate_inputs 方法来校验输入参数是否完整
    >> 这个方法会检查所有预期的键是否存在于输入字典中，如果有任何缺失的键，则抛出一个错误。
    self._validate_inputs(inputs)
    >> 最后，返回处理后的输入字典。
    return inputs
```

```python
>> 主要职责是验证和准备链的输出数据
>> 还会将执行过程中的相关信息保存到类内部的运行时存储（内存）
def prep_outputs(
    self,
    inputs: Dict[str, str],  >> 这应是包含原始输入的字典类型的数据，可能包括从内存加载的附加输入
    outputs: Dict[str, str], >> 包含初始链输出的字典，需要被进一步处理和验证
    return_only_outputs: bool = False,   >> 指示是否只返回输出数据,  False，则输入也将添加到最终的输出中(key_value)
) -> Dict[str, str]:
    """Validate and prep outputs."""
    >> 调用 _validate_outputs 方法对 outputs 参数进行校验，确保所有的输出数据都是完整和有效的。
    self._validate_outputs(outputs)
    if self.memory is not None:
        >> 如果 Chain 类具有内存功能, 那么会把 inputs 和 outputs 的上下文信息保存到内存中
        >> 这样可以让我们在后续的执行或调试中回溯这次的运行过程。
        self.memory.save_context(inputs, outputs)
    >> 根据 return_only_outputs 参数决定返回值：如果为 True，那么只返回 outputs；
    if return_only_outputs:
        return outputs
    >> 否则，将输入和输出合并后一起返回。
    else:
        return {**inputs, **outputs}
```
    

**call**：此方法执行链，并处理一些回调配置和输入/输出处理。

```python
def __call__(
    self,
    inputs: Union[Dict[str, Any], Any], >> inputs: 预期为包含输入数据的字典或单一值, 字典的键应与 Chain.input_keys 对应，除非预期的输入将从链的内部存储加载
    return_only_outputs: bool = False,  >> True k-v, False v 
    callbacks: Callbacks = None,        >> 回调函数列表，它们将在此次链运行过程中调用，作为构建时指定的回调函数之外的额外操作
) -> Dict[str, Any]:                    >> 一个包含预期输出的字典，其键应存在于 Chain.output_keys 中定义的键集合。
    """Run the logic of this chain and add to output if desired.

    Args:
        inputs: Dictionary of inputs, or single input if chain expects
            only one param.
        return_only_outputs: boolean for whether to return only outputs in the
            response. If True, only new keys generated by this chain will be
            returned. If False, both input keys and new keys generated by this
            chain will be returned. Defaults to False.
            
        

    """
    >> 将输入数据进行预处理，以确保它们满足链的实际运行需求。
    inputs = self.prep_inputs(inputs)
    >> 配置 回调管理器 ,以便在链的运行过程中执行特定的操作。
    callback_manager = CallbackManager.configure(
        callbacks, self.callbacks, self.verbose
    )
    new_arg_supported = inspect.signature(self._call).parameters.get("run_manager")
    >> 使用已配置的 回调管理器 启动链的运行。
    run_manager = callback_manager.on_chain_start(
        {"name": self.__class__.__name__},
        inputs,
    )
    try:
        >> 尝试执行 _call 方法（这是在各个子类中根据具体需求定义的方法）以实现链的实际操作
        >> 如果在运行过程中出现任何异常，将由   回调管理器   捕获并处理。
        outputs = (
            self._call(inputs, run_manager=run_manager)
            if new_arg_supported
            else self._call(inputs)
        )
    except (KeyboardInterrupt, Exception) as e:
        >> 如果在运行过程中出现任何异常，将由   回调管理器   捕获并处理。
        run_manager.on_chain_error(e)
        raise e
    >> 在链运行结束后，通过回调管理器处理关闭事件，例如清理资源，记录日志等。
    run_manager.on_chain_end(outputs)
    >> 对链运行生成的输出进行预处理，以满足返回格式的要求
    >> 返回值: 一个包含预期输出的字典，其键应存在于 Chain.output_keys 中定义的键集合。
    return self.prep_outputs(inputs, outputs, return_only_outputs)
```

![](./imgs/call_method.png)

![](./imgs/call_time_seq.png)

**apply**: 该方法在列表中的所有输入上调用链

**run**：对<font color=blue>单字符串输出执行链的便利方法</font>, 不同于 Chain.call 的是，它只适用于返回单个字符串输出的链。

## 二: LLMChain

1. **语言学习模型（LLM）查询的类**<br>
2. **构建复杂的自然语言处理应用**: 如对话系统、问答系统等

### 属性

1. prompt: BasePromptTemplate: 该属性存储一个对象，负责根据给定的输入生成相应的提示。
2. llm: BaseLanguageModel：这是实际执行查询的语言模型对象。
3. output_key: str = "text": 字符串，定义了**输出字典**中结果的键，默认为<font color=blue>"text"</font>。

### 方法

1. `_call()`: 该方法调用 generate 方法，并从其响应中创建输出

![](./imgs/LLMChain_call.png)

```python
def _call(
    self,
    inputs: Dict[str, Any],
    run_manager: Optional[CallbackManagerForChainRun] = None,
) -> Dict[str, str]:
    >> 1. _call方法首先调用generate方法
    response = self.generate([inputs], run_manager=run_manager)
    >> 4. 最后，_call使用create_outputs创建输出，返回输出列表的第一个元素。
    return self.create_outputs(response)[0]


def generate(
    self,
    input_list: List[Dict[str, Any]],
    run_manager: Optional[CallbackManagerForChainRun] = None,
) -> LLMResult:
    """Generate LLM result from inputs."""
    >> 2. 然后在generate方法中，它会调用prep_prompts来准备提示
    prompts, stop = self.prep_prompts(input_list, run_manager=run_manager)
    >> 3. 然后使用llm.generate_prompt生成结果
    return self.llm.generate_prompt(
        prompts, stop, callbacks=run_manager.get_child() if run_manager else None
    )
```


2. `apply()`: 此方法使用语言学习模型的 generate 方法产生结果，以实现高效运行。
3. `generate()`: 该方法根据输入生成语言学习模型的结果
4. `prep_prompts()`: 这个函数负责根据输入准备相应的提示。
5. `create_outputs()`: 从模型响应中创建并格式化输出
6. `from_string()` <font color=blue>类方法</font> 接受一个语言学习模型和一个模板字符串作为参数，然后返回一个配置好的 LLMChain 实例。

### 对比Chain的改动

1. 新增了一个**llm**属性,这个是**Chain**没有的
2. input_keys 和 output_keys: 这两个属性是 LLMChain 从 Chain 类中继承并重写的
-----
```python
@property
@abstractmethod
def input_keys(self) -> List[str]:
    """Input keys this chain expects."""

@property
@abstractmethod
def output_keys(self) -> List[str]:
    """Output keys this chain expects."""
```
---

```python
@property
def input_keys(self) -> List[str]:
    """Will be whatever keys the prompt expects.

    :meta private:
    """
    >> 返回self.prompt.input_variables的值
    return self.prompt.input_variables

@property
def output_keys(self) -> List[str]:
    """Will always return text key.

    :meta private:
    """
    >> 返回最终的输出结果
    return [self.output_key]
```

3. `_call()`: 这是从 Chain 类中覆盖的方法，它是执行链对象的核心接口。在 LLMChain 中，该方法被定制化以适应特定的语言学习模型调用方式。
4. `generate()`: <font color=blue>新增的方法</font>: 它负责根据给定的输入数据生成语言学习模型的结果。

5. `prep_prompts()`: <font color=blue>新增的方法</font>: 用于根据输入准备相应的提示。

> * 这个函数的主要目的是根据输入列表创建语言模型的提示列表 <br>
> * 同时检查所有输入中的"stop"值是否相同。

```python
def prep_prompts(
    self,
    input_list: List[Dict[str, Any]],
    run_manager: Optional[CallbackManagerForChainRun] = None,
) -> Tuple[List[PromptValue], Optional[List[str]]]:
    """Prepare prompts from inputs."""
    stop = None
    if "stop" in input_list[0]:
        stop = input_list[0]["stop"]
    prompts = []
    for inputs in input_list:
        selected_inputs = {k: inputs[k] for k in self.prompt.input_variables}
        prompt = self.prompt.format_prompt(**selected_inputs)
        _colored_text = get_colored_text(prompt.to_string(), "green")
        _text = "Prompt after formatting:\n" + _colored_text
        if run_manager:
            run_manager.on_text(_text, end="\n", verbose=self.verbose)
        if "stop" in inputs and inputs["stop"] != stop:
            raise ValueError(
                "If `stop` is present in any inputs, should be present in all."
            )
        prompts.append(prompt)
    return prompts, stop
```

6. `create_outputs()`<font color=blue>新增的方法</font>: 负责根据模型的响应创建格式化的输出。

> * 一个转换函数，其主要目的是将语言学习模型（LLM）生成的结果 (LLMResult) 格式化为期望的输出格式。

```python
def create_outputs(self, response: LLMResult) -> List[Dict[str, str]]:
    """Create outputs from response."""
    return [
        # Get the text of the top generated string.
        {self.output_key: generation[0].text}
        for generation in response.generations
    ]
```




8. `from_string()`: 这是一个类方法，设计为外部调用接口。它接受一个语言学习模型和一个模板字符串作为参数，然后返回一个配置好的 LLMChain 实例。